Documentation [To complete]

In [ ]:
# Import necessary libraries

import os
import random
import time
import shutil
import json

In [3]:
# we import the already know optimal strategy for tic-tac-toe 

# Load optimal move book from JSON
try:
    with open("tictactoe_book.json") as f:
        BOOK = json.load(f)
except FileNotFoundError:
    BOOK = {}
    print("Warning: tictactoe_book.json not found, falling back to Minimax.")

# AI randomness probability
AI_RANDOMNESS = 0.2  # 20% chance to make a random move

In [4]:
def clear():
    # Clear the console screen based on the operating system
    os.system('cls' if os.name == 'nt' else 'clear')

In [5]:
def no_possible_win(board):
    """
    Check if there is no possible winning condition left on the board.

    Args:
        board (list of list of str): A 3x3 tic-tac-toe board where each cell contains 'X', 'O', or ''.

    Returns:
        bool: True if no winning condition is possible, False otherwise.
    """
    win_cond = [
        (0,0,0,1,0,2), (1,0,1,1,1,2), (2,0,2,1,2,2),
        (0,0,1,0,2,0), (0,1,1,1,2,1), (0,2,1,2,2,2),
        (0,0,1,1,2,2), (0,2,1,1,2,0)
    ]
    for x1, y1, x2, y2, x3, y3 in win_cond:
        line = [board[x1][y1], board[x2][y2], board[x3][y3]]
        if not ('X' in line and 'O' in line):
            return False
    return True

In [6]:
def print_board(board, score, players, show_numbers):
    """
    Clears the console and prints the current state of the tic-tac-toe board with player names and scores.

    Args:
        board (list of list of str): The 3x3 game board. Each cell contains 'X', 'O', or ' '.
        score (dict): A dictionary with players' names as keys and their current scores as values.
        players (list of str): List containing names of the two players. Expected order: [player1, player2].
        show_numbers (bool): If True, empty cells are annotated with their corresponding cell number (1-9).

    Returns:
        None
    """
    # Clear the console screen (using the clear() function defined in cell index 3)
    clear()
    
    # Get the terminal width to format the output centrally
    width = shutil.get_terminal_size().columns
    
    # Create header and score display strings
    header = f"{players[0]} vs {players[1]}"
    score_line = f"Score: {players[0]} {score[players[0]]} - {score[players[1]]} {players[1]}"
    
    # Print the header centered and the score right-aligned
    print(header.center(width))
    print(score_line.rjust(width))
    print("\n")
    
    # Prepare the display board with numbers in empty cells if show_numbers is True
    display = []
    for i in range(3):
        row = []
        for j in range(3):
            # If showing numbers and cell is empty (' '), display cell number starting at 1
            if show_numbers and board[i][j] == ' ':
                row.append(f"({i*3+j+1})")
            else:
                row.append(board[i][j])
        display.append(row)
    
    # Define cell width and separator line for board display
    cell_width = 7
    sep_line = "+" + "+".join(["=" * cell_width] * 3) + "+"
    
    # Print each row of the board with appropriate formatting
    for row in display:
        # Center each item in the cell
        line = "|" + "|".join(item.center(cell_width) for item in row) + "|"
        pad = " " * ((width - len(line)) // 2)
        print(pad + sep_line)
        print(pad + line)
    
    # Print the bottom separator line
    pad = " " * ((width - len(sep_line)) // 2)
    print(pad + sep_line)
    
    # Display additional help message
    print("\nType 'esc' to return to menu at any time.\n")

In [8]:
def check_win(board, player):
    """
    Check if the given player has achieved a win on the tic-tac-toe board.

    A win occurs when the player's symbol occupies all positions in any of the predefined winning combinations.

    Args:
        board (list of list of str): A 3x3 tic-tac-toe board. Each cell contains 'X', 'O', or ' '.
        player (str): The player's symbol ('X' or 'O') to check for a win.

    Returns:
        bool: True if the player has won, False otherwise.
    """
    # Define the eight possible winning conditions as tuples of cell indices.
    win_cond = [
        (0,0, 0,1, 0,2),  # Top row
        (1,0, 1,1, 1,2),  # Middle row
        (2,0, 2,1, 2,2),  # Bottom row
        (0,0, 1,0, 2,0),  # Left column
        (0,1, 1,1, 2,1),  # Middle column
        (0,2, 1,2, 2,2),  # Right column
        (0,0, 1,1, 2,2),  # Diagonal from top-left to bottom-right
        (0,2, 1,1, 2,0)   # Diagonal from top-right to bottom-left
    ]
    
    # Iterate through each winning condition
    for x1, y1, x2, y2, x3, y3 in win_cond:
        # Check if all three cells in the current winning condition have the player's symbol.
        if board[x1][y1] == board[x2][y2] == board[x3][y3] == player:
            return True  # Winning condition met
    
    # If none of the winning conditions are met, return False.
    return False

In [7]:
def board_full(board):
    # This function checks if the tic-tac-toe board is full.
    # It returns True if every cell in every row is not an empty space, and False otherwise.
    return all(cell != ' ' for row in board for cell in row)

In [9]:
def get_move(board, player):
    """
    Prompt the player to enter their move on the tic-tac-toe board.

    The function ensures the move is valid (i.e., within the range 1-9, corresponds to an empty cell, 
    or the player can type 'esc' to exit). If the input is invalid, the player is prompted again.

    Args:
        board (list of list of str): The 3x3 game board. Each cell contains 'X', 'O', or ' '.
        player (str): The name or symbol of the player making the move.

    Returns:
        tuple: A tuple (row, column) representing the player's chosen cell, or 'esc' if the player exits.
    """
    while True:
        move = input(f"{player}, enter your move (1-9): ")
        if move.lower() == 'esc':
            return 'esc'
        if move.isdigit() and 1 <= int(move) <= 9:
            r, c = divmod(int(move) - 1, 3)
            if board[r][c] == ' ':
                return r, c
            else:
                print("Cell is taken!")
        else:
            print("Invalid input.")

In [10]:
# Minimax with depth and alpha-beta
def minimax(board, depth, is_maximizing, alpha, beta):
    if check_win(board, 'O'):
        return 10 - depth
    if check_win(board, 'X'):
        return depth - 10
    if board_full(board):
        return 0

    priority = [(1,1)] + [(0,0),(0,2),(2,0),(2,2)] + [(0,1),(1,0),(1,2),(2,1)]
    if is_maximizing:
        max_eval = -float('inf')
        for i,j in [(i,j) for i,j in priority if board[i][j] == ' ']:
            board[i][j] = 'O'
            eval = minimax(board, depth+1, False, alpha, beta)
            board[i][j] = ' '
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for i,j in [(i,j) for i,j in priority if board[i][j] == ' ']:
            board[i][j] = 'X'
            eval = minimax(board, depth+1, True, alpha, beta)
            board[i][j] = ' '
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval


In [11]:
def best_move(board):
    best_val = -float('inf')
    best = None
    priority = [(1,1)] + [(0,0),(0,2),(2,0),(2,2)] + [(0,1),(1,0),(1,2),(2,1)]
    for i,j in [(i,j) for i,j in priority if board[i][j] == ' ']:
        board[i][j] = 'O'
        move_val = minimax(board, 0, False, -float('inf'), float('inf'))
        board[i][j] = ' '
        if move_val > best_val:
            best_val = move_val
            best = (i, j)
    return best

In [12]:
def ai_move(board):
    if random.random() < AI_RANDOMNESS:
        # random move to add unpredictability
        return random.choice([(i, j) for i in range(3) for j in range(3) if board[i][j] == ' '])
    key = "".join(cell if cell != ' ' else '-' for row in board for cell in row)
    if key in BOOK:
        idx = BOOK[key]
        return divmod(idx, 3)
    mv = best_move(board)
    if mv:
        return mv
    return random.choice([(i, j) for i in range(3) for j in range(3) if board[i][j] == ' '])

In [13]:
def play_round(players, single_mode, score, show_numbers):
    board = [[' ']*3 for _ in range(3)]
    turn = 0
    while True:
        print_board(board, score, players, show_numbers)
        current = players[turn%2]
        symbol = 'X' if turn%2==0 else 'O'
        if single_mode and current=="AI":
            time.sleep(0.5)
            r,c = ai_move(board)
        else:
            mv = get_move(board, current)
            if mv=='esc':
                return 'esc'
            r,c = mv
        board[r][c]=symbol
        if check_win(board, symbol):
            print_board(board, score, players, show_numbers)
            print(f"{current} wins!")
            score[current]+=1
            time.sleep(1)
            return
        if board_full(board) or no_possible_win(board):
            print_board(board, score, players, show_numbers)
            print("It's a tie!")
            time.sleep(1)
            return
        turn+=1

In [14]:

def game_loop(single_mode):
    clear()
    if single_mode:
        p1=input("Enter your name: ")
        players=[p1,"AI"]
    else:
        p1=input("Player 1 name: ")
        p2=input("Player 2 name: ")
        players=[p1,p2]
    while True:
        ch=input("Numbered grid? (y/n): ")
        if ch.lower()=='esc': return
        if ch.lower() in('y','n'): show_numbers=ch.lower()=='y'; break
    while True:
        rd=input(f"{players[0]}, how many rounds? ")
        if rd.lower()=='esc': return
        if rd.isdigit() and int(rd)>0: rounds=int(rd); break
    score={players[0]:0,players[1]:0}
    for _ in range(rounds):
        if play_round(players,single_mode,score,show_numbers)=='esc': return
    clear()
    print_board([[' ']*3]*3, score, players, show_numbers)
    print(f"Final: {players[0]} {score[players[0]]}-{score[players[1]]} {players[1]}")
    if score[players[0]]>score[players[1]]: print(f"{players[0]} wins!")
    elif score[players[0]]<score[players[1]]: print(f"{players[1]} wins!")
    else: print("Draw!")
    input("Enter to menu...")

In [15]:
def main_menu():
    while True:
        clear()
        print("="*50); print(" ★ TIC-TAC-TOE TERMINAL EDITION ★ ".center(50)); print("="*50)
        print("1. Single Player"); print("2. Pass & Play"); print("3. Exit")
        print("esc to menu.")
        c=input("Choice: ")
        if c=='1': game_loop(True)
        elif c=='2': game_loop(False)
        elif c=='3': break
        else: time.sleep(1)

In [16]:

if __name__=="__main__":
    main_menu()

         ★ TIC-TAC-TOE TERMINAL EDITION ★         
1. Single Player
2. Pass & Play
3. Exit
esc to menu.
                                    1 vs AI                                     
                                                               Score: 1 0 - 0 AI


                           +=======+=======+=======+
                           |  (1)  |  (2)  |  (3)  |
                           +=======+=======+=======+
                           |  (4)  |  (5)  |  (6)  |
                           +=======+=======+=======+
                           |  (7)  |  (8)  |  (9)  |
                           +=======+=======+=======+

Type 'esc' to return to menu at any time.

                                    1 vs AI                                     
                                                               Score: 1 0 - 0 AI


                           +=======+=======+=======+
                           |   X   |  (2)  |  (3)  |
                           +=======+=======+===

KeyboardInterrupt: Interrupted by user